# Model Fit in Linear Regression - Lab

## Introduction
In this lab, you'll learn how to evaluate your model results and you'll learn how to select the appropriate features using stepwise selection.

## Objectives
You will be able to:
* Use stepwise selection methods to determine the most important features for a model
* Use recursive feature elimination to determine the most important features for a model

## The Ames Housing Data once more

In [1]:
import pandas as pd
import numpy as np

ames = pd.read_csv('ames.csv')

continuous = ['LotArea', '1stFlrSF', 'GrLivArea', 'SalePrice']
categoricals = ['BldgType', 'KitchenQual', 'SaleType', 'MSZoning', 'Street', 'Neighborhood']

ames_cont = ames[continuous]

# log features
log_names = [f'{column}_log' for column in ames_cont.columns]

ames_log = np.log(ames_cont)
ames_log.columns = log_names

# normalize (subract mean and divide by std)

def normalize(feature):
    return (feature - feature.mean()) / feature.std()

ames_log_norm = ames_log.apply(normalize)

# one hot encode categoricals
ames_ohe = pd.get_dummies(ames[categoricals], prefix=categoricals, drop_first=True)

preprocessed = pd.concat([ames_log_norm, ames_ohe], axis=1)

## Perform stepwise selection

The function for stepwise selection is copied below. Use this provided function on your preprocessed Ames Housing data.

In [2]:
import statsmodels.api as sm

def stepwise_selection(X, y, 
                       initial_list=[], 
                       threshold_in=0.01, 
                       threshold_out = 0.05, 
                       verbose=True):
    """ 
    Perform a forward-backward feature selection 
    based on p-value from statsmodels.api.OLS
    Arguments:
        X - pandas.DataFrame with candidate features
        y - list-like with the target
        initial_list - list of features to start with (column names of X)
        threshold_in - include a feature if its p-value < threshold_in
        threshold_out - exclude a feature if its p-value > threshold_out
        verbose - whether to print the sequence of inclusions and exclusions
    Returns: list of selected features 
    Always set threshold_in < threshold_out to avoid infinite looping.
    See https://en.wikipedia.org/wiki/Stepwise_regression for the details
    """
    included = list(initial_list)
    while True:
        changed=False
        # forward step
        excluded = list(set(X.columns)-set(included))
        new_pval = pd.Series(index=excluded, dtype='float')
        for new_column in excluded:
            model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included+[new_column]]))).fit()
            new_pval[new_column] = model.pvalues[new_column]
        best_pval = new_pval.min()
        if best_pval < threshold_in:
            best_feature = new_pval.idxmin()
            included.append(best_feature)
            changed=True
            if verbose:
                print('Add  {:30} with p-value {:.6}'.format(best_feature, best_pval))

        # backward step
        model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included]))).fit()
        # use all coefs except intercept
        pvalues = model.pvalues.iloc[1:]
        worst_pval = pvalues.max() # null if pvalues is empty
        if worst_pval > threshold_out:
            changed=True
            #worst_feature = pvalues.argmax()
            worst_feature = pvalues.idxmax()
            included.remove(worst_feature)
            if verbose:
                print('Drop {:30} with p-value {:.6}'.format(worst_feature, worst_pval))
        if not changed:
            break
    return included

In [3]:
# Your code here
x = preprocessed.drop('SalePrice_log', axis=1)
y = preprocessed['SalePrice_log']

res = stepwise_selection(x, y, verbose=False)
res

['GrLivArea_log',
 'KitchenQual_TA',
 '1stFlrSF_log',
 'KitchenQual_Fa',
 'Neighborhood_OldTown',
 'KitchenQual_Gd',
 'Neighborhood_Edwards',
 'Neighborhood_IDOTRR',
 'LotArea_log',
 'Neighborhood_NridgHt',
 'BldgType_Duplex',
 'Neighborhood_NAmes',
 'Neighborhood_SWISU',
 'Neighborhood_BrkSide',
 'Neighborhood_Sawyer',
 'Neighborhood_NoRidge',
 'Neighborhood_StoneBr',
 'Neighborhood_MeadowV',
 'SaleType_New',
 'Neighborhood_BrDale',
 'MSZoning_RM',
 'MSZoning_RL',
 'MSZoning_FV',
 'MSZoning_RH',
 'Neighborhood_NWAmes',
 'Neighborhood_Mitchel',
 'Neighborhood_SawyerW']

### Build the final model again in Statsmodels

In [4]:
import statsmodels.api as sm

x_int = sm.add_constant(x[res])
model = sm.OLS(y, x_int).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          SalePrice_log   R-squared:                       0.835
Model:                            OLS   Adj. R-squared:                  0.832
Method:                 Least Squares   F-statistic:                     269.0
Date:                Sat, 05 Dec 2020   Prob (F-statistic):               0.00
Time:                        18:35:34   Log-Likelihood:                -754.40
No. Observations:                1460   AIC:                             1565.
Df Residuals:                    1432   BIC:                             1713.
Df Model:                          27                                         
Covariance Type:            nonrobust                                         
========================================================================================
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
const                   -0.2174      0.164     -1.323      0.186      -0.540       0.105
GrLivArea_log            0.3694      0.015     24.477      0.000       0.340       0.399
KitchenQual_TA          -0.7020      0.055    -12.859      0.000      -0.809      -0.595
1stFlrSF_log             0.1445      0.015      9.645      0.000       0.115       0.174
KitchenQual_Fa          -1.0372      0.087    -11.864      0.000      -1.209      -0.866
Neighborhood_OldTown    -0.8625      0.063    -13.615      0.000      -0.987      -0.738
KitchenQual_Gd          -0.4021      0.050     -8.046      0.000      -0.500      -0.304
Neighborhood_Edwards    -0.7019      0.048    -14.530      0.000      -0.797      -0.607
Neighborhood_IDOTRR     -0.8583      0.097     -8.855      0.000      -1.048      -0.668
LotArea_log              0.1096      0.015      7.387      0.000       0.081       0.139
Neighborhood_NridgHt     0.3854      0.057      6.809      0.000       0.274       0.496
BldgType_Duplex         -0.4073      0.061     -6.678      0.000      -0.527      -0.288
Neighborhood_NAmes      -0.3763      0.038     -9.981      0.000      -0.450      -0.302
Neighborhood_SWISU      -0.6263      0.089     -7.020      0.000      -0.801      -0.451
Neighborhood_BrkSide    -0.5641      0.066     -8.493      0.000      -0.694      -0.434
Neighborhood_Sawyer     -0.4026      0.055     -7.342      0.000      -0.510      -0.295
Neighborhood_NoRidge     0.4347      0.070      6.221      0.000       0.298       0.572
Neighborhood_StoneBr     0.4538      0.087      5.226      0.000       0.283       0.624
Neighborhood_MeadowV    -0.6622      0.118     -5.592      0.000      -0.895      -0.430
SaleType_New             0.1483      0.044      3.388      0.001       0.062       0.234
Neighborhood_BrDale     -0.4733      0.123     -3.839      0.000      -0.715      -0.231
MSZoning_RM              1.0820      0.147      7.363      0.000       0.794       1.370
MSZoning_RL              0.9916      0.156      6.356      0.000       0.686       1.298
MSZoning_FV              1.2052      0.165      7.284      0.000       0.881       1.530
MSZoning_RH              0.8503      0.189      4.490      0.000       0.479       1.222
Neighborhood_NWAmes     -0.2055      0.054     -3.837      0.000      -0.311      -0.100
Neighborhood_Mitchel    -0.1943      0.065     -3.004      0.003      -0.321      -0.067
Neighborhood_SawyerW    -0.1666      0.058     -2.862      0.004      -0.281      -0.052
==============================================================================
Omnibus:                      295.535   Durbin-Watson:                   1.965
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1270.571
Skew:                          -0.903   Prob(JB):                    1.26e-276
Kurtosis

## Use Feature ranking with recursive feature elimination

Use feature ranking to select the 5 most important features

In [5]:
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression

linreg = LinearRegression()
selector = RFE(linreg, n_features_to_select=5)
selector = selector.fit(x, y)
selector.support_ 

array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
        True,  True,  True,  True, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False,  True, False, False, False, False, False, False, False,
       False, False])

Fit the linear regression model again using the 5 selected columns

In [6]:
selected_columns = x.columns[selector.support_]
linreg.fit(x[selected_columns], y)

LinearRegression()

Now, predict $\hat y$ using your model. You can use `.predict()` in scikit-learn. 

In [7]:
ym = linreg.predict(x[selected_columns])

Now, using the formulas of R-squared and adjusted R-squared below, and your Python/numpy knowledge, compute them and contrast them with the R-squared and adjusted R-squared in your statsmodels output using stepwise selection. Which of the two models would you prefer?

$SS_{residual} = \sum (y - \hat{y})^2 $

$SS_{total} = \sum (y - \bar{y})^2 $

$R^2 = 1- \dfrac{SS_{residual}}{SS_{total}}$

$R^2_{adj}= 1-(1-R^2)\dfrac{n-1}{n-p-1}$

In [8]:
SS_Residual = np.sum((y-ym)**2)
SS_Total = np.sum((y-np.mean(y))**2)
r_squared = 1-(float(SS_Residual))/SS_Total
adjusted_r_squared = 1-(1-r_squared)*(len(y)-1)/(len(y)-x[selected_columns].shape[1]-1)

# r_squared is 0.239434  
# adjusted_r_squared is 0.236818
r_squared,adjusted_r_squared

(0.23943418177114217, 0.2368187559863112)

## Level up (Optional)

- Perform variable selection using forward selection, using this resource: https://planspace.org/20150423-forward_selection_with_statsmodels/. Note that this time features are added based on the adjusted R-squared!
- Tweak the code in the `stepwise_selection()` function written above to just perform forward selection based on the p-value 

In [9]:
import statsmodels.formula.api as smf

def forward_selected(data, response):
    """Linear model designed by forward selection.

    Parameters:
    -----------
    data : pandas DataFrame with all possible predictors and response

    response: string, name of response column in data

    Returns:
    --------
    model: an "optimal" fitted statsmodels linear model
           with an intercept
           selected by forward selection
           evaluated by adjusted R-squared
    """
    remaining = set(data.columns)
    remaining.remove(response)
    selected = []
    current_score, best_new_score = 0.0, 0.0
    while remaining and current_score == best_new_score:
        scores_with_candidates = []
        for candidate in remaining:
            formula = "{} ~ {} + 1".format(response, ' + '.join(selected + [candidate]))
            score = smf.ols(formula, data).fit().rsquared_adj
            scores_with_candidates.append((score, candidate))
        scores_with_candidates.sort()
        best_new_score, best_candidate = scores_with_candidates.pop()
        if current_score < best_new_score:
            remaining.remove(best_candidate)
            selected.append(best_candidate)
            current_score = best_new_score
    formula = "{} ~ {} + 1".format(response, ' + '.join(selected))
    model = smf.ols(formula, data).fit()
    return model

In [10]:
preprocessed.rename(columns={'1stFlrSF_log':'stFlrSF_log'}, inplace=True)

In [11]:
model = forward_selected(preprocessed, 'SalePrice_log')
model.rsquared_adj

0.8344715728761254

In [12]:
model.model.formula

'SalePrice_log ~ GrLivArea_log + KitchenQual_TA + stFlrSF_log + KitchenQual_Fa + Neighborhood_OldTown + KitchenQual_Gd + Neighborhood_Edwards + Neighborhood_IDOTRR + LotArea_log + Neighborhood_NridgHt + BldgType_Duplex + Neighborhood_NAmes + Neighborhood_SWISU + Neighborhood_BrkSide + Neighborhood_Sawyer + Neighborhood_NoRidge + Neighborhood_Somerst + Neighborhood_StoneBr + Neighborhood_MeadowV + SaleType_New + SaleType_WD + Neighborhood_BrDale + MSZoning_RM + MSZoning_RL + MSZoning_FV + MSZoning_RH + Neighborhood_NWAmes + Neighborhood_Mitchel + Neighborhood_SawyerW + SaleType_Con + Neighborhood_ClearCr + BldgType_2fmCon + SaleType_ConLD + Neighborhood_Veenker + Street_Pave + BldgType_Twnhs + Neighborhood_Timber + 1'

## Summary
Great! You practiced your feature selection skills by applying stepwise selection and recursive feature elimination to the Ames Housing dataset! 